In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import utils, learn

# import models
from model_zoo import residual_connectomics_model, residual_connectomics_model2

In [2]:
filename = 'processed_dataset.hdf5'
#data_path = 'D:/Dropbox/TFconnect'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf#[:10000,:,:,:]
y_train = ltf#[:10000,:]
X_valid = dtf_crossval#[:5000,:,:,:]
y_valid = ltf_crossval#[:5000,:]

CPU times: user 448 ms, sys: 1.02 s, total: 1.47 s
Wall time: 1.47 s


In [ ]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model2.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# set output file paths
results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = '1d_version_residual2'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 32)
layer3: conv1_batch
(?, 2, 326, 32)
layer4: conv1_active
(?, 2, 326, 32)
layer5: conv1_dropout
(?, 2, 326, 32)
layer6: resid1_1resid
(?, 2, 326, 32)
layer7: resid1_1resid_norm
(?, 2, 326, 32)
layer8: resid1_1resid_active
(?, 2, 326, 32)
layer9: resid1_2resid
(?, 2, 326, 32)
layer10: resid1_2resid_norm
(?, 2, 326, 32)
layer11: resid1_resid_sum
(?, 2, 326, 32)
layer12: resid1_resid
(?, 2, 326, 32)
layer13: resid1_batch
(?, 2, 326, 32)
layer14: resid1_dropout
(?, 2, 326, 32)
layer15: conv2
(?, 1, 322, 64)
layer16: conv2_batch
(?, 1, 322, 64)
layer17: conv2_active
(?, 1, 322, 64)
layer18: conv2_dropout
(?, 1, 322, 64)
layer19: resid2_1resid
(?, 1, 322, 64)
layer20: resid2_1resid_norm
(?, 1, 322, 64)
layer21: resid2_1resid_active
(?, 1, 322, 64)
layer22: resid

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.8, 'keep_prob_dense': 0.5, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=200, 
                    patience=20, verbose=2, shuffle=True)

Epoch 1 out of 200 
[=======                       ] 23.7% -- time=234s -- loss=0.44725 -- accuracy=82.46%  

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.9, 'keep_prob_dense': 0.7, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}

nntrainer.set_best_parameters()
learn.train_minibatch(nntrainer, data, batch_size=1000, num_epochs=100, 
                    patience=10, verbose=2, shuffle=True)

In [ ]:


def validation_score(nntrainer, data_path):

    filename = 'valideval_dataset.hdf5'
    dataset = h5py.File(os.path.join(data_path,filename),'r')
    group_name = ['valid_data']
    val_dat = np.array(dataset['/'+group_name[0]+'/vs_valid'])
    val_lbl = np.array(dataset['/'+group_name[0]+'/label_valid'])

    fragLen = 330
    N = 14
    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil(float(val_dat.shape[1] - fragLen)/N).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0
    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print('\r' + 'X'*(a//100))

        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

        # Count the number of traces in each batch
        cnt = 0

        for b in range(val_dat.shape[0]):

            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()

                cnt += 1

            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0

            cnt_ += 1

        # Run batch through network
        test = {'inputs': im_eval, 'keep_prob_conv': 1, 'keep_prob_dense': 1, 'is_training': False}
        pred_stop = nntrainer.get_activations(test, layer='output')[:,0]
        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
            cnt_u += 1        


    from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
    fpr, tpr, thresholds = roc_curve(true_lbl, pred_lbl)
    wrk = auc(fpr, tpr)
    print(wrk)
    return wrk

nntrainer.set_best_parameters()
validation_score(nntrainer, data_path)

In [ ]:

def test_prediction(nntrainer, data_path):

    filename = 'competition_dataset_downsampled.hdf5'
    dataset = h5py.File(os.path.join(data_path,filename),'r')
    group_name = ['competition_data']
    val_dat = np.array(dataset['/'+group_name[0]+'/realval'])
    val_lbl = np.array(dataset['/'+group_name[0]+'/realtest'])

    fragLen = 330
    N = 14
    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil(float(val_dat.shape[1] - fragLen)/N).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0
    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print('\r' + 'X'*(a//100))

        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

        # Count the number of traces in each batch
        cnt = 0

        for b in range(val_dat.shape[0]):

            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()

                cnt += 1

            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0

            cnt_ += 1

        # Run batch through network
        test = {'inputs': im_eval, 'keep_prob_conv': 1, 'keep_prob_dense': 1, 'is_training': False}
        pred_stop = nntrainer.get_activations(test, layer='output')[:,0]
        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
            cnt_u += 1        
    return pred_lbl


nntrainer.set_best_parameters()
pred_lbl = test_prediction(nntrainer, data_path)
